In [5]:
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.stats import linregress

import featureEngineering

# Step 1: Load EEG data from CSV file
df = pd.read_csv('downsampled_final_125Hz.csv', usecols=lambda column: column not in ['Unnamed: 0.1', 'Unnamed: 0'])

# Ensure data is sorted by participant_id, epoch, and timestep to avoid issues in feature extraction
df = df.sort_values(by=['participant_id', 'epoch', 'timestep']).reset_index(drop=True)

# Define EEG channel columns (assuming 'channel_1' to 'channel_8' are the relevant EEG channels)
eeg_channels = [f'channel_{i}' for i in range(1, 9)]

# Define sampling rate (Hz)
fs = 125  # Assuming 125Hz as the sampling rate

# Step 2: Define a function to extract features from each epoch
def extract_features_by_epoch(df, eeg_channels, fs=125):
    feature_list = []

    # PCA
    pca = featureEngineering.FeatureEngineering(df, eeg_channels)
    df = pca.apply_pca(1)
    df = pca.apply_pca_per_subject(1)

    # Group data by participant and epoch
    grouped = df.groupby(['participant_id', 'epoch'])

    for (participant_id, epoch), group in grouped:
        features = {}

        # Extract participant ID, epoch, and timestep range (from the first to the last timestep in the epoch)
        features['participant_id'] = participant_id
        features['epoch'] = epoch
        features['timestep_start'] = group['timestep'].iloc[0]
        features['timestep_end'] = group['timestep'].iloc[-1]

        # 1. Time-Domain Features (Max, Min, Mean, Amplitude Range)
        for channel in eeg_channels:
            channel_values = group[channel].values
            features[f'{channel}_max'] = np.max(channel_values)
            features[f'{channel}_min'] = np.min(channel_values)
            features[f'{channel}_mean'] = np.mean(channel_values)
            features[f'{channel}_amplitude_range'] = np.max(channel_values) - np.min(channel_values)

        # 2. Frequency-Domain Features (Alpha, Beta, Gamma Power using Welch method)
        for channel in eeg_channels:
            nperseg = min(len(group[channel].values), fs)
            freqs, psd = welch(group[channel].values, fs=fs, nperseg=nperseg)
            alpha_power = np.sum(psd[(freqs >= 8) & (freqs <= 12)])
            beta_power = np.sum(psd[(freqs >= 13) & (freqs <= 30)])
            gamma_power = np.sum(psd[(freqs >= 30) & (freqs <= 50)])
            
            features[f'{channel}_alpha_power'] = alpha_power
            features[f'{channel}_beta_power'] = beta_power
            features[f'{channel}_gamma_power'] = gamma_power

        # 3. Fast Fourier Transform (FFT) Features
        for channel in eeg_channels:
            channel_values = group[channel].values
            fft_values = np.fft.fft(channel_values)
            fft_freqs = np.fft.fftfreq(len(fft_values), d=1/fs)

            # Consider only the positive frequencies
            positive_fft_values = fft_values[:len(fft_values) // 2]
            positive_fft_freqs = fft_freqs[:len(fft_freqs) // 2]

            # FFT features
            total_power = np.sum(np.abs(positive_fft_values) ** 2)  # Total power in frequency domain
            dominant_freq = positive_fft_freqs[np.argmax(np.abs(positive_fft_values))]  # Frequency with the maximum amplitude

            features[f'{channel}_fft_total_power'] = total_power
            features[f'{channel}_fft_dominant_frequency'] = dominant_freq

        # 3.5 Add PCA
        features[f'pca_universal'] = df['pca_universal1']
        features[f'pca_per_subject'] = df['pca_1']

        # 4. Signal Slope (Dynamic Feature)
        for channel in eeg_channels:
            slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(channel_values)), channel_values)
            features[f'{channel}_slope'] = slope

        # 5. Inter-Channel Differences (Spatial Features)
        for i in range(len(eeg_channels)):
            for j in range(i + 1, len(eeg_channels)):
                channel_i = group[eeg_channels[i]].values
                channel_j = group[eeg_channels[j]].values
                features[f'{eeg_channels[i]}_{eeg_channels[j]}_diff'] = np.mean(channel_i - channel_j)

        # Add the trigger value (assuming we're taking the trigger from the last timestep in the epoch)
        features['trigger'] = group['trigger'].iloc[-1]

        # Append extracted features for this epoch
        feature_list.append(features)

    # Convert list of feature dicts into a DataFrame
    return pd.DataFrame(feature_list)

# Step 3: Apply feature extraction by epoch
features_df = extract_features_by_epoch(df, eeg_channels, fs=fs)
features_df['pca_universal'] = pd.to_numeric(features_df['pca_universal'], errors='coerce')
features_df['pca_per_subject'] = pd.to_numeric(features_df['pca_per_subject'], errors='coerce')


# Step 4: Save extracted features to a new CSV file
features_df.to_csv('extracted_features.csv', index=False)

# Step 5: Optional - print first few rows of the resulting features DataFrame to verify
print(features_df.head())

print(features_df['pca_universal'].dtypes)
print(features_df['pca_per_subject'].dtypes)


e:\University\ConfusedMatrixP300\featureEngineering.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-25.57116426 -18.14214261 -14.01784787 ... -36.26507467 -20.09630506
 -59.49384606]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.data_table.loc[self.data_table['participant_id'].eq(participant_id), 'pca_1'] = new_values[:, 0]


   participant_id  epoch  timestep_start  timestep_end  channel_1_max  \
0               1      1     1303.574029   1356.425971      20.649432   
1               1      2     1348.574029   1401.425971      18.230486   
2               1      3     1391.574029   1444.425971      -1.165833   
3               1      4     1432.574029   1485.425971       8.848805   
4               1      5     1479.574029   1532.425971      13.029421   

   channel_1_min  channel_1_mean  channel_1_amplitude_range  channel_2_max  \
0     -13.017932        4.339864                  33.667364      13.707586   
1     -15.943325       -3.003764                  34.173811      13.845616   
2     -13.979912       -6.508167                  12.814079       2.827503   
3      -9.891811       -0.109365                  18.740615       6.261853   
4     -11.033756        1.093455                  24.063177      17.102832   

   channel_2_min  ...  channel_4_channel_6_diff  channel_4_channel_7_diff  \
0     -18.88350

| **Feature Name**                        | **Description**                                                                             |
|-----------------------------------------|---------------------------------------------------------------------------------------------|
| `participant_id`                        | Unique identifier for the participant, used to differentiate signal data from different individuals. |
| `timestep_start`                        | The starting timestep of the sliding window, marking the beginning of the data segment.      |
| `timestep_end`                          | The ending timestep of the sliding window, marking the end of the data segment.              |
| `channel_X_max`                         | Maximum value of channel `X` within the current window, capturing the peak signal.          |
| `channel_X_min`                         | Minimum value of channel `X` within the current window, capturing the lowest signal.        |
| `channel_X_mean`                        | Mean value of channel `X` within the current window, representing the overall signal level. |
| `channel_X_amplitude_range`             | Difference between the max and min values of channel `X`, showing the signal's amplitude range. |
| `channel_X_alpha_power`                 | Power of the alpha band (8-12 Hz) for channel `X`, linked to relaxation and attention.      |
| `channel_X_beta_power`                  | Power of the beta band (13-30 Hz) for channel `X`, associated with cognitive activity and focus. |
| `channel_X_gamma_power`                 | Power of the gamma band (30-50 Hz) for channel `X`, related to high-level cognitive processes. |
| `channel_X_fft_total_power`             | Total power in the frequency domain of channel `X`, representing the signal's overall energy. |
| `channel_X_fft_dominant_frequency`      | Dominant frequency of channel `X`, indicating the most significant frequency component in the signal. |
| `channel_X_slope`                       | Slope of the signal for channel `X` in the current window, indicating the trend (rising or falling) of the signal. |
| `channel_X_channel_Y_diff`              | Average difference between signals of channel `X` and channel `Y`, showing inter-channel relationships. |
| `trigger`                               | Event label or trigger value, indicating whether a specific event (e.g., P300) occurred within the window. |


In [ ]:
df

   participant_id  timestep_start  timestep_end  channel_1_max  channel_1_min  \
0               5    59491.157032  59601.507948      14.575491     -15.777908   
1               5    59491.157032  59601.507948      14.575491     -15.777908   
2               5    59491.157032  59601.507948      14.575491     -15.777908   
3               5    59491.157032  59601.507948      14.575491     -15.777908   
4               5    59491.157032  59601.507948      14.575491     -15.777908   

   channel_1_mean  channel_1_amplitude_range  channel_2_max  channel_2_min  \
0       -2.437404                  30.353399       3.202417     -18.177887   
1       -2.437404                  30.353399       3.202417     -18.177887   
2       -2.437404                  30.353399       3.202417     -18.177887   
3       -2.437404                  30.353399       3.202417     -18.177887   
4       -2.437404                  30.353399       3.202417     -18.177887   

   channel_2_mean  ...  channel_4_channel_5_

In [2]:
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.stats import linregress

# Step 1: Load EEG data from CSV file
df = pd.read_csv('downsampled_final_125Hz.csv', usecols=lambda column: column not in ['Unnamed: 0.1', 'Unnamed: 0'])

df = df.sort_values(by=['participant_id', 'timestep']).reset_index(drop=True)

# Define EEG channel columns (assuming 'channel_1' to 'channel_8' are the relevant EEG channels)
eeg_channels = [f'channel_{i}' for i in range(1, 9)]

# Define sampling rate (Hz)
fs = 125  # Assuming 125Hz as the sampling rate

# Define window size (e.g., 500ms) and step size (e.g., 100ms)
window_size = int(0.5 * fs)  # 500ms window, which equals 62 samples at 125Hz
step_size = int(0.1 * fs)    # 100ms step, equals 12 samples

# Step 2: Define a function to extract features from each window
def extract_features_with_sliding_window(df, eeg_channels, fs=125, window_size=62, step_size=12):
    feature_list = []

    # Sliding window over the entire dataset
    for i in range(0, len(df) - window_size + 1, step_size):
        window_df = df.iloc[i:i + window_size]  # Extract the current window of data
        features = {}

        # Extract participant ID and timestep (from the first row of the window)
        features['participant_id'] = window_df['participant_id'].iloc[0]
        features['timestep_start'] = window_df['timestep'].iloc[0]
        features['timestep_end'] = window_df['timestep'].iloc[-1]

        # 1. Time-Domain Features (Max, Min, Mean, Amplitude Range)
        for channel in eeg_channels:
            channel_values = window_df[channel].values
            features[f'{channel}_max'] = np.max(channel_values)
            features[f'{channel}_min'] = np.min(channel_values)
            features[f'{channel}_mean'] = np.mean(channel_values)
            features[f'{channel}_amplitude_range'] = np.max(channel_values) - np.min(channel_values)

        # 2. Frequency-Domain Features (Alpha, Beta, Gamma Power using Welch method)
        for channel in eeg_channels:
            nperseg = min(len(window_df[channel].values), fs)
            freqs, psd = welch(window_df[channel].values, fs=fs, nperseg=nperseg)
            alpha_power = np.sum(psd[(freqs >= 8) & (freqs <= 12)])
            beta_power = np.sum(psd[(freqs >= 13) & (freqs <= 30)])
            gamma_power = np.sum(psd[(freqs >= 30) & (freqs <= 50)])
            
            features[f'{channel}_alpha_power'] = alpha_power
            features[f'{channel}_beta_power'] = beta_power
            features[f'{channel}_gamma_power'] = gamma_power
        
        # 3. Fast Fourier Transform (FFT) Features
        for channel in eeg_channels:
            channel_values = window_df[channel].values
            fft_values = np.fft.fft(channel_values)
            fft_freqs = np.fft.fftfreq(len(fft_values), d=1/fs)

            # Consider only the positive frequencies
            positive_fft_values = fft_values[:len(fft_values) // 2]
            positive_fft_freqs = fft_freqs[:len(fft_freqs) // 2]

            # FFT features
            total_power = np.sum(np.abs(positive_fft_values) ** 2)  # Total power in frequency domain
            dominant_freq = positive_fft_freqs[np.argmax(np.abs(positive_fft_values))]  # Frequency with the maximum amplitude

            features[f'{channel}_fft_total_power'] = total_power
            features[f'{channel}_fft_dominant_frequency'] = dominant_freq

        # 4. Signal Slope (Dynamic Feature)
        for channel in eeg_channels:
            channel_values = window_df[channel].values
            slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(channel_values)), channel_values)
            features[f'{channel}_slope'] = slope

        # 5. Inter-Channel Differences (Spatial Features)
        for i in range(len(eeg_channels)):
            for j in range(i + 1, len(eeg_channels)):
                channel_i = window_df[eeg_channels[i]].values
                channel_j = window_df[eeg_channels[j]].values
                features[f'{eeg_channels[i]}_{eeg_channels[j]}_diff'] = np.mean(channel_i - channel_j)

        # Add the trigger value (assuming we're taking the trigger from the last timestep in the window)
        features['trigger'] = window_df['trigger'].iloc[-1]

        # Append extracted features for this window
        feature_list.append(features)

    # Convert list of feature dicts into a DataFrame
    return pd.DataFrame(feature_list)

# Step 3: Apply feature extraction with sliding window
features_df = extract_features_with_sliding_window(df, eeg_channels, fs=fs, window_size=window_size, step_size=step_size)

# Step 4: Save extracted features to a new CSV file
features_df.to_csv('extracted_features_by_window.csv', index=False)

# Step 5: Optional - print first few rows of the resulting features DataFrame to verify
print(features_df.head())


   participant_id  timestep_start  timestep_end  channel_1_max  channel_1_min  \
0               1     1303.574029   1414.492052      20.649432     -15.943325   
1               1     1329.500000   1437.089260      20.649432     -15.943325   
2               1     1350.325328   1454.524488      15.628820     -15.943325   
3               1     1371.492052   1478.089260       8.848805     -15.943325   
4               1     1394.089260   1498.448961       8.848805     -15.943325   

   channel_1_mean  channel_1_amplitude_range  channel_2_max  channel_2_min  \
0       -1.006528                  36.592758      13.845616     -18.883505   
1       -0.803566                  36.592758      13.845616     -15.320706   
2       -4.242605                  31.572146      11.781255     -15.320706   
3       -4.381019                  24.792130       6.261853     -15.320706   
4       -3.913574                  24.792130      11.396095     -15.320706   

   channel_2_mean  ...  channel_4_channel_6_